# Introduction
This script gets a list of all the pods that do not have resources set as part of their deployment.
Pods in the `redhat-rhoam` namespace is selected.

The only external tool this script requires is `oc` and it should be logged into a cluster you wish to investigate.
There is no error handling done around the oc calling some debug may be required if the json models are not loaded.



# Raw Data
Get the values for all the pods on the cluster.
This is all the pod in a running or complete state.

In [1]:
from subprocess import run

raw_data = run(["oc", "get", "pods", "-A", "-o", "json"], capture_output=True)

# Data models

In [2]:
from pydantic import BaseModel
from typing import List, Optional

class Metadata(BaseModel):
    name: Optional[str] = None
    namespace: Optional[str] = None
    
class Limits(BaseModel):
    cpu: Optional[str] = None
    memory: Optional[str] = None

class Resources(BaseModel):
    limits: Optional[Limits] = None
    requests: Optional[Limits] = None
    
class Containers(BaseModel):
    resources: Optional[Resources] = None
    image: Optional[str]
    name: Optional[str]
    
class Spec(BaseModel):
    containers: List[Containers]
    
class CR(BaseModel):
    metadata: Metadata
    spec: Spec

class CRList(BaseModel):
    items: List[CR]
    
class Unit(BaseModel):
    back_ref: Optional[CR]
    resources: Optional[Resources] = None
    image: Optional[str]
    name: Optional[str]


# Load Data
Load a list of all the pods found on the cluster

In [3]:
import json
data = json.loads(raw_data.stdout)
data = CRList(**data)
print(f"Number of pods found {len(data.items)}")

Number of pods found 572


# Filter Data
Filter only pods in the RHOAM namespaces

In [4]:
namespace_prefix = "redhat-rhoam"
items = []

# item = data.items

for item in data.items:
    if item.metadata.namespace.startswith(namespace_prefix):
        items.append(item)
        
print(f"Number of pods in RHOAM name spaces: {len(items)}")

Number of pods in RHOAM name spaces: 94


Switch view to use the containers in a pod over the pods.
A pod may have many containers with in it, which may have no values set.

Sort the containers in to list which have resources set or not.


In [5]:
units = []

for item in items:
    for container in item.spec.containers:
        unit = Unit()
        unit.name = container.name
        unit.image = container.image
        unit.resources = container.resources
        unit.back_ref = item
        units.append(unit)

print(len(units))

115


In [6]:
no_limits = []
has_limits = []

for item in units: 
    if item.resources.limits is None or item.resources.requests is None:
        no_limits.append(item)
    else:
        has_limits.append(item)

In [7]:
print(f"pods with no limits {len(no_limits)}")
print(f"pods with limits {len(has_limits)}")

pods with no limits 71
pods with limits 44


Group the limit issues


In [8]:
no_resources = []
no_resource_limits = []
no_resource_requests = []
unknown = []

for item in no_limits:
    if item.resources.limits is None and item.resources.requests is None:
        no_resources.append(item)
    elif item.resources.limits is None:
        no_resource_limits.append(item)
    elif item.resources.requests is None:
        no_resource_requests.append(item)
    else:
        unknown.append(item)


In [9]:
print(f"Number of pods with both sets of resources missing {len(no_resources)}")
print(f"Number of pods with limits missing {len(no_resource_limits)}")
print(f"Number of pods with request limits missing {len(no_resource_requests)}")
if len(unknown) > 0:
    print(f"Some pods seem to have issues beeing added to the correct list. Number of these pods are {len(unknown)}")

Number of pods with both sets of resources missing 51
Number of pods with limits missing 20
Number of pods with request limits missing 0


# Explore data


## No resources set
Get the list of pods that have no resources set.
Pods that are collect as unknown are not worked on here.

In [10]:
from tabulate import tabulate

header = ["Namespace", "Pod Name", "Container Name"]
data = []

for container in no_resources:
    data.append((container.back_ref.metadata.namespace, container.back_ref.metadata.name, container.name))

table = tabulate(data, header)
print(table)

Namespace                                   Pod Name                                                    Container Name
------------------------------------------  ----------------------------------------------------------  -----------------------
redhat-rhoami-3scale-operator               threescale-operator-controller-manager-v2-7ffc85b7c7-ql4k7  manager
redhat-rhoami-3scale                        apicast-production-1-deploy                                 deployment
redhat-rhoami-3scale                        apicast-production-3-dd2qh                                  envoy-sidecar
redhat-rhoami-3scale                        apicast-production-3-deploy                                 deployment
redhat-rhoami-3scale                        apicast-production-3-j58kr                                  envoy-sidecar
redhat-rhoami-3scale                        apicast-staging-1-deploy                                    deployment
redhat-rhoami-3scale                        apicast-staging-

## No Resource Limits

Get the list of pods that have no resource limits set

In [11]:
from tabulate import tabulate

header = ["Namespace", "Pod Name", "Container Name"]
data = []

for container in no_resource_limits:
    data.append((container.back_ref.metadata.namespace, container.back_ref.metadata.name, container.name))

table = tabulate(data, header)
print(table)

Namespace                                   Pod Name                                                         Container Name
------------------------------------------  ---------------------------------------------------------------  -------------------------
redhat-rhoami-3scale-operator               3c4048818f3d4f4b848ee3446c31fe30e8dbadffa5d25713bfafb308f364f4n  extract
redhat-rhoami-3scale-operator               rhmi-registry-cs-7pmdf                                           registry-server
redhat-rhoami-cloud-resources-operator      671e4af6bcbfc4898da1669f40663e3579fac50fe4430f8f3850fef45aj9lhw  extract
redhat-rhoami-cloud-resources-operator      rhmi-registry-cs-ldlm2                                           registry-server
redhat-rhoami-customer-monitoring-operator  rhmi-registry-cs-9p4vm                                           configmap-registry-server
redhat-rhoami-marin3r-operator              rhmi-registry-cs-p4hnh                                           configmap-reg

## No Resource Requessts
List the pods that don't have any resources requests set.

In [12]:
from tabulate import tabulate

header = ["Namespace", "Pod Name", "Container Name"]
data = []

for container in no_resource_requests:
    data.append((container.back_ref.metadata.namespace, container.back_ref.metadata.name, container.name))

table = tabulate(data, header)
print(table)

Namespace    Pod Name    Container Name
-----------  ----------  ----------------


## Unknown state
This is a list of containers that could not be classed into the above groups.
It is the failsafe.
This list is expected to be always empty

In [ ]:
from tabulate import tabulate

header = ["Namespace", "Pod Name", "Container Name"]
data = []

for container in unknown:
    data.append((container.back_ref.metadata.namespace, container.back_ref.metadata.name, container.name))

table = tabulate(data, header)
print(table)